In [ ]:
!pip install pdfplumber
!pip install pymupdf

In [ ]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving Jn1.pdf to Jn1.pdf
Saving Jn2.pdf to Jn2.pdf
Saving Jn3.pdf to Jn3.pdf
Saving Jn4.pdf to Jn4.pdf
Saving Jn5.pdf to Jn5.pdf
Saving Jn6.pdf to Jn6.pdf
Saving Jn7.pdf to Jn7.pdf
Saving Jn8.pdf to Jn8.pdf
Saving Jn9.pdf to Jn9.pdf
Saving Jn10.pdf to Jn10.pdf
Saving Jn11.pdf to Jn11.pdf
Saving Jn12.pdf to Jn12.pdf
Saving Jn13.pdf to Jn13.pdf
Saving Jn14.pdf to Jn14.pdf
Saving Jn15.pdf to Jn15.pdf
Saving Jn16.pdf to Jn16.pdf
Saving Jn17.pdf to Jn17.pdf
Saving Jn19.pdf to Jn19.pdf
Saving Jn20.pdf to Jn20.pdf
Saving Jn21.pdf to Jn21.pdf
Saving Jn22.pdf to Jn22.pdf


# Run

In [ ]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
import fitz
import shutil

In [ ]:
output_folder = "output_pages"
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

count = 0
os.makedirs(output_folder, exist_ok=True)
for filename in uploaded.keys():
    doc = fitz.open(filename)
    for i, page in enumerate(doc):
        text = page.get_text("text")
        lines = text.splitlines()
        if lines and "HCM Signalized Intersection Capacity Analysis" in lines[0]:
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=i, to_page=i)
            output_path = os.path.join(output_folder, f"HCM_Signalized_{count+1}.pdf")
            new_doc.save(output_path)
            new_doc.close()
            count += 1

In [ ]:
def get_nth_numeric_value(data, n):
    numeric_values = []
    for item in data:
        try:
            num = float(item)
            numeric_values.append(num)
        except (ValueError, TypeError):
            continue
    return numeric_values[n - 1] if len(numeric_values) >= n else None

def extract_lines_below_pattern(lines, pattern, offset=2, range_length=1):
    for i, line in enumerate(lines):
        if pattern.strip().lower() in line.strip().lower():
            start_index = i + offset
            end_index = start_index + range_length
            return [l.strip() for l in lines[start_index:end_index] if start_index < len(lines)]
    return []

In [ ]:
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS', 'W_Movement','W_Delay','W_V/C','W_LOS'] )
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    all_text = []
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            text = page.extract_text( layout=True)
            if text:
                all_text.append(text)
    full_text = '\n'.join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)

    node= data[4][9:30]
    Scenario= data[4][32:]
    stripped_lines = extract_lines_below_pattern(data, "Intersection Summary", offset=1, range_length=2)
    abc = stripped_lines[0].split(" ")
    control_delay = get_nth_numeric_value(abc, 2)
    level_of_service = stripped_lines[0].split(" ")[-1]
    v_c_ratio = stripped_lines[1].split(" ")[-1]
    line_1 = extract_lines_below_pattern(data, "Movement", offset=0, range_length=1)
    line_2 = extract_lines_below_pattern(data, "Intersection Summary", offset=-8, range_length=6)
    lines = line_1 + line_2
    ref = [i for i in range(len(lines[0][17:])) if (i == 0 or lines[0][17:][i-1] == ' ') and lines[0][17:][i] != ' ']
    df_row = []
    for k in [0,1,-2,-1]:
        starts = [i for i in range(len(lines[k][17:])) if (i == 0 or lines[k][17:][i-1] == ' ') and lines[k][17:][i] != ' ']
        text = lines[k][17:]
        values = []
        for i, start in enumerate(starts):
            end = starts[i + 1] if i + 1 < len(starts) else len(text)
            val = text[start:end].strip()
            values.append((start, val))
        row = ['' for _ in range(len(ref))]
        for start_idx, val in values:
            for i, r in enumerate(ref):
                if abs(start_idx - r) <= 2:
                    row[i] = val
                    break
        df_row.append(row)
    df = pd.DataFrame(df_row, columns=df_row[0])
    try:
        delay_row = pd.to_numeric(df.iloc[2], errors='coerce')
        max_col = delay_row.idxmax()
        vc = df.loc[1, max_col] if 1 in df.index and max_col in df.columns else 'NA'
        delay = df.loc[2, max_col] if 2 in df.index and max_col in df.columns else 'NA'
        los = df.loc[3, max_col] if 3 in df.index and max_col in df.columns else 'NA'

    except (IndexError, KeyError, ValueError):
        vc = delay = los = 'NA'
    new_row = {
        'Scenario': Scenario,
        'Node': node,
        'V/C':v_c_ratio,
        'Delay': control_delay,
        'LOS': level_of_service,
        'W_Movement': max_col,
        'W_Delay': delay,
        'W_V/C': vc,
        'W_LOS': los
    }

    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

<ipython-input-324-22ff4c34f87e>:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)
<ipython-input-324-22ff4c34f87e>:45: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_col = delay_row.idxmax()


# change Output file Name as per your choice "Synchro_Report.xlsx"

In [ ]:
Output

,Scenario,Node,V/C,Delay,LOS,W_Movement,W_Delay,W_V/C,W_LOS
0,...,Junction 118:,0.13,20.3,C,SBT,20.6,0.17,C
1,...,Junction 152:,0.75,13.5,B,SBL,26.2,0.65,C
2,...,Junction 153:,0.71,20.3,C,WBT,30.8,0.66,C
3,...,JN 15:,0.86,9.2,A,NEL,53.0,0.57,D
4,...,JN 16:,0.32,11.5,B,NWL,17.9,0.04,B
...,...,...,...,...,...,...,...,...,...
89,...,JN 22:,1.00,58.8,E,EBL,84.5,1.04,F
90,...,Junction 56:,0.27,21.8,C,WBL,22.8,0.39,C
91,...,JN 8:,0.84,30.3,C,SWL,63.9,0.36,E
92,...,Junction 46:,0.14,19.5,B,WBL,21.0,0.03,C


In [ ]:
Output.to_excel('Synchro_Report.xlsx', index=False)